In [1]:
# Imports

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [2]:
# Data adquisition    

def get_students_katas(students_csv='input/students.csv', katas_csv='input/katas.csv'): 
    s = pd.read_csv('input/students.csv').set_index('username')
    k = pd.read_csv('input/katas.csv').set_index('slug')
    return s, k

students, katas = get_students_katas()
katas

,date,minutes
slug,,
regexp-fun-number-1-when-i-miss-few-days-of-gym,2019-02-01T16:20:21.241Z,60
deodorant-evaporator,2019-02-01T16:20:21.241Z,60
ordered-count-of-characters,2019-02-13T10:00:00.241Z,45
reverse-or-rotate,2019-02-15T13:00:00.241Z,60


In [3]:
# Data transformation

# transform minutes to int
katas.minutes = katas.minutes.astype('int64')

# transform date to datetime
katas.date = pd.to_datetime(katas.date,infer_datetime_format=True)
katas.head()

,date,minutes
slug,,
regexp-fun-number-1-when-i-miss-few-days-of-gym,2019-02-01 16:20:21.241000+00:00,60
deodorant-evaporator,2019-02-01 16:20:21.241000+00:00,60
ordered-count-of-characters,2019-02-13 10:00:00.241000+00:00,45
reverse-or-rotate,2019-02-15 13:00:00.241000+00:00,60


In [4]:
   
# add minutes to each row combining 2 columns
katas['limit'] = katas.apply(lambda row: row['date'] + pd.Timedelta(minutes=row['minutes']), axis=1)
katas.limit = pd.to_datetime(katas.limit,infer_datetime_format=True)
katas.dtypes
katas.head()

,date,minutes,limit
slug,,,
regexp-fun-number-1-when-i-miss-few-days-of-gym,2019-02-01 16:20:21.241000+00:00,60,2019-02-01 17:20:21.241000+00:00
deodorant-evaporator,2019-02-01 16:20:21.241000+00:00,60,2019-02-01 17:20:21.241000+00:00
ordered-count-of-characters,2019-02-13 10:00:00.241000+00:00,45,2019-02-13 10:45:00.241000+00:00
reverse-or-rotate,2019-02-15 13:00:00.241000+00:00,60,2019-02-15 14:00:00.241000+00:00


In [5]:
df = pd.DataFrame(index=students.index, columns=katas.index)
df.head()

slug,regexp-fun-number-1-when-i-miss-few-days-of-gym,deodorant-evaporator,ordered-count-of-characters,reverse-or-rotate
username,,,,
paulapr,NaN,NaN,NaN,NaN
Almugs,NaN,NaN,NaN,NaN
hector-moreno,NaN,NaN,NaN,NaN
potacho,NaN,NaN,NaN,NaN
carlosmd14,NaN,NaN,NaN,NaN


In [6]:
# https://www.codewars.com/api/v1/users/Livia Canet/code-challenges/completed

# https://www.codewars.com/kata/

def get_completed_by_user(user, 
                  host='https://www.codewars.com/api/v1/users/', 
                  path='/code-challenges/completed'):
    url = host + user + path
    resp = requests.get(url)
    return resp.json()

In [7]:
# tarda un rato

# Crea diccionario con usuario: resultado de la API
completed = dict([(u, get_completed_by_user(u)) for u in students.index])
# completed

In [9]:

# dictionary username: lista de katas incluidas en katas.index
# completed_katas = dict([(k, [e for e in v['data'] if e['slug'] in katas.index]) for k, v in completed.items()] )

# dictionary username: lista de katas completadas
completed_katas_dict = dict([(k, v['data']) for k, v in completed.items()] )
completed_katas_dict

{'paulapr': [{'id': '56b5afb4ed1f6d5fb0000991',
   'name': 'Reverse or rotate?',
   'slug': 'reverse-or-rotate',
   'completedLanguages': ['python'],
   'completedAt': '2019-02-15T14:23:21.428Z'},
  {'id': '57a6633153ba33189e000074',
   'name': 'Ordered Count of Characters',
   'slug': 'ordered-count-of-characters',
   'completedLanguages': ['python'],
   'completedAt': '2019-02-13T09:11:58.899Z'},
  {'id': '5720a81309e1f9b232001c5b',
   'name': 'RegExp Fun #1 - When I miss few days of gym',
   'slug': 'regexp-fun-number-1-when-i-miss-few-days-of-gym',
   'completedLanguages': ['python'],
   'completedAt': '2019-02-01T16:20:21.241Z'},
  {'id': '5506b230a11c0aeab3000c1f',
   'name': 'Deodorant Evaporator',
   'slug': 'deodorant-evaporator',
   'completedLanguages': ['python'],
   'completedAt': '2019-01-31T10:23:30.056Z'},
  {'id': '541da001259d9ca85d000688',
   'name': 'Arithmetic List!',
   'slug': 'arithmetic-list',
   'completedLanguages': ['python'],
   'completedAt': '2019-01-25T0

In [13]:
'''
employee.groupby(["Department"]).agg(lambda x: (x["Salary"].max() - x["Salary"].min()))["Salary"]
kickstarter["goal_met"] = np.where(kickstarter.usd_pledged_real > kickstarter.usd_goal_real,1,0)
'''

def user_completed_kata_intime(user, kata_slug, completed): 
    for data in completed[user]: 
        if data['slug'] == kata_slug: 
            return True
    return False


def user_time_kata(user, slug, completed, katas): 
    for data in completed[user]: 
        if data['slug'] == slug: 
            return data['completedAt']
    return False

def kata_in_time(user, slug, df_time, katas): 
    return katas.loc[slug]['limit'] > df_time.loc[user][slug]
    

df_finish = df.copy()
df_time = df.copy()
df_in_time = df.copy()
for user in students.index: 
    for slug in katas.index: 
        df_finish.loc[user, slug] = user_completed_kata_intime(user, slug, completed_katas_dict)
        df_time.loc[user, slug] = user_time_kata(user, slug, completed_katas_dict, katas)
        # df_time = pd.to_datetime(df_time, infer_datetime_format=True)
        # df_in_time.loc[user, slug] = kata_in_time(user, slug, df_time, katas)

display(df_finish.head())
display(df_time.head())
display(df_in_time.head())

df_time.dtypes


slug,regexp-fun-number-1-when-i-miss-few-days-of-gym,deodorant-evaporator,ordered-count-of-characters,reverse-or-rotate
username,,,,
paulapr,True,True,True,True
Almugs,False,True,True,False
hector-moreno,True,True,True,True
potacho,True,True,False,True
carlosmd14,True,True,True,True


slug,regexp-fun-number-1-when-i-miss-few-days-of-gym,deodorant-evaporator,ordered-count-of-characters,reverse-or-rotate
username,,,,
paulapr,2019-02-01T16:20:21.241Z,2019-01-31T10:23:30.056Z,2019-02-13T09:11:58.899Z,2019-02-15T14:23:21.428Z
Almugs,False,2019-01-31T10:07:42.203Z,2019-02-13T09:22:22.716Z,False
hector-moreno,2019-02-01T09:20:36.608Z,2019-01-31T09:50:30.396Z,2019-02-13T09:32:00.389Z,2019-02-15T14:28:12.748Z
potacho,2019-02-01T10:20:48.105Z,2019-01-31T09:33:19.252Z,False,2019-02-15T14:36:18.055Z
carlosmd14,2019-02-01T09:40:03.013Z,2019-01-31T09:15:42.108Z,2019-02-13T09:12:50.216Z,2019-02-15T12:28:29.372Z


slug,regexp-fun-number-1-when-i-miss-few-days-of-gym,deodorant-evaporator,ordered-count-of-characters,reverse-or-rotate
username,,,,
paulapr,NaN,NaN,NaN,NaN
Almugs,NaN,NaN,NaN,NaN
hector-moreno,NaN,NaN,NaN,NaN
potacho,NaN,NaN,NaN,NaN
carlosmd14,NaN,NaN,NaN,NaN


slug
regexp-fun-number-1-when-i-miss-few-days-of-gym    object
deodorant-evaporator                               object
ordered-count-of-characters                        object
reverse-or-rotate                                  object
dtype: object

In [ ]:
df_time.head()
df_time.dtypes

In [ ]:
def kata_in_time(df, katas, slug=''): 
    res = []
    for student in df.index: 
        # print(df.loc[student][slug] - )
        print(type(df.loc[student][slug] ) )
        # print(timedelta(minutes=int(katas.loc[slug]['minutes'])))
        # print(df.loc[student][slug] - timedelta(minutes=int(katas.loc[slug]['minutes'])))
    
kata_in_time(df_time, katas, 'ordered-count-of-characters')    